# AutoEncoders

## Importing the libraries

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset

In [2]:
movies = pd.read_csv("movies.dat",sep='::',header=None,engine='python',encoding='latin-1')
users = pd.read_csv("users.dat",encoding='latin-1',header=None,engine='python',sep='::')
ratings = pd.read_csv("ratings.dat",engine='python',encoding='latin-1',header=None,sep='::')

## Preparing the training set and the test set

In [3]:
training_set = pd.read_csv("u1.base",delimiter='\t')
training_set = np.array(training_set,dtype='int')
test_set= pd.read_csv('u1.test',delimiter='\t')
test_set = np.array(test_set,dtype='int')

## Getting the number of users and movies

In [4]:
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))

## Converting the data into an array with users in lines and movies in columns

In [5]:
def convert(data) :
    new_data = list()
    for id_users in range(1,nb_users+1) :
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors

In [6]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the archtecture of the Neural Network

In [8]:
class SAE(nn.Module) :
    def __init__(self,):
        super(SAE,self).__init__()
        self.fc1 = nn.Linear(nb_movies,20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20,nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self,x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(),lr=0.01,weight_decay=0.5)

## Training the SAE

In [9]:
nb_epoch = 200
for epoch in range(1,nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0) # adds and additional dimension to the data(batch_size),making it compatible with the model input.
        target = input.clone()   # since we're trying to reconstruct the input, the input is being cloned.
        if torch.sum(target.data>0)>0 :  # counts non-zero entries in the user's rating data.
            output = sae(input)
            target.require_grad = False # target tensor shouldn't be calculating grads during backpropagation, since it's used only for comparison , not for opmization.
            output[target==0] = 0 # sets the values of unrated movies(target=0) to 0.. cuz weight update and backpropagation don't count those kinda shit.so it is just for saving up some memories.
            loss = criterion(output,target)
            mean_corrector = nb_movies/float(torch.sum(target.data>0)+1e-10) # normalizing the loss by the number of non-zero movies. it is either for mean_squared blah2 shit or for preventing bias.
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+ ' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(1.7709)
epoch: 2 loss: tensor(1.0967)
epoch: 3 loss: tensor(1.0533)
epoch: 4 loss: tensor(1.0385)
epoch: 5 loss: tensor(1.0306)
epoch: 6 loss: tensor(1.0267)
epoch: 7 loss: tensor(1.0238)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0205)
epoch: 10 loss: tensor(1.0195)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0183)
epoch: 13 loss: tensor(1.0177)
epoch: 14 loss: tensor(1.0176)
epoch: 15 loss: tensor(1.0174)
epoch: 16 loss: tensor(1.0171)
epoch: 17 loss: tensor(1.0166)
epoch: 18 loss: tensor(1.0165)
epoch: 19 loss: tensor(1.0163)
epoch: 20 loss: tensor(1.0159)
epoch: 21 loss: tensor(1.0160)
epoch: 22 loss: tensor(1.0160)
epoch: 23 loss: tensor(1.0158)
epoch: 24 loss: tensor(1.0157)
epoch: 25 loss: tensor(1.0156)
epoch: 26 loss: tensor(1.0156)
epoch: 27 loss: tensor(1.0150)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0131)
epoch: 30 loss: tensor(1.0115)
epoch: 31 loss: tensor(1.0094)
epoch: 32 loss: tensor(1.0095)
epoch: 33 loss: t

## Testing the SAE

In [13]:
test_loss = 0
s = 0.
for id_user in range(nb_users) :
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data>0) >0 :
        output = sae(input)
        target.require_grad = False
        output[target==0]=0
        loss =criterion(output,target)
        mean_corrector = nb_movies/float(torch.sum(target.data>0)+1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print("test loss: "+ str(test_loss/s))

test loss: tensor(0.9570)
